In [21]:
import pandas as pd
import re
import yahooquery as yq
import datetime
import numpy as np

In [22]:
# Leggi il file CSV
df = pd.read_csv('/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/tesi28.csv', sep=';')
df


,ID,Ragione sociale,Ticker symbol,ISIN number,Numero CCIAA,Indice corrente\n2022,Indice corrente\n2021,Indice corrente\n2020,Indice corrente\n2019,Indice corrente\n2018,...,Debt/Equity ratio\n%\n2022,Debt/Equity ratio\n%\n2021,Debt/Equity ratio\n%\n2020,Debt/Equity ratio\n%\n2019,Debt/Equity ratio\n%\n2018,Debt/Equity ratio\n%\n2017,Debt/Equity ratio\n%\n2016,Debt/Equity ratio\n%\n2015,Debt/Equity ratio\n%\n2014,Debt/Equity ratio\n%\n2013
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,"1,58","1,57","1,70","1,46","1,67",...,"0,95","0,86","1,29","1,04","0,94","0,92","0,89","0,80","0,71","0,82"
1,2.0,PIAGGIO & C. S.P.A.,PIA,IT0003073266,PI0134077,"0,98","0,95","0,79","0,83","0,97",...,"0,96","1,12","1,09","0,91","0,82","0,64","0,92","0,78","0,74","0,95"
2,3.0,BREMBO S.P.A.,BRE,IT0005252728,BG0134667,"1,55","1,64","1,99","1,32","1,14",...,"0,76","0,84","0,98","0,55","0,50","0,63","0,66","0,69","1,01","1,00"
3,4.0,DANIELI & C. OFFICINE MECCANICHE S.P.A. IN FOR...,DAN,IT0000076502,UD0084904,"0,80","0,82","0,84","0,84","0,82",...,"0,45","0,76","0,73","0,43","0,37","0,51","0,51","0,31","0,18","0,25"
4,5.0,BRUNELLO CUCINELLI S.P.A.,BC,IT0004764699,PG0165936,"1,53","1,58","1,21","1,81","1,92",...,"0,44","0,48","0,72","0,48","0,27","0,29","0,33","0,42","0,48","0,33"
5,6.0,CENTRALE DEL LATTE D'ITALIA S.P.A.,CLI,IT0003023980,TO0520409,"0,90","0,95","0,99","0,97","1,30",...,"1,26","1,36","1,78","0,36","0,21","0,23","0,20","0,31","0,20","0,23"
6,7.0,ELICA S.P.A.,ELC,IT0003404214,AN0063006,"1,16","1,10","1,41","1,12","1,09",...,"0,82","1,13","1,30","0,92","0,96","1,07","0,75","0,72","0,75","0,74"
7,8.0,GPI S.P.A.,GPI,IT0005221517,TN0189428,"2,12","1,48","1,84","1,65","1,76",...,"0,92","1,20","1,49","1,28","1,37","0,63","0,77","2,22","0,89","1,35"
8,9.0,SERVIZI ITALIA S.P.A.,SRI,IT0003814537,PR0185104,"0,61","0,55","0,52","0,60","0,68",...,"1,06","0,99","1,06","0,92","0,66","0,63","0,61","0,57","0,53","0,75"
9,10.0,A.L.A. SOCIETA' PER AZIONI,ALA,IT0005446700,NA0803780,"1,98","2,31","1,61","1,27","1,17",...,"0,92","0,58","1,03","1,02","1,14","1,38","1,27","1,23","2,86","1,01"


## scarica i valori di borsa per ogni ticker, per ogni anno

In [23]:
tickers = df['Ticker symbol']
valori_borsa = []
tickers_non_scaricati=[]

# Scarica i dati storici per ciascun ticker e anno
for ticker in tickers:
    yq_ticker = yq.Ticker(ticker+".mi")
    for year in range(2022, 2012, -1):
        # Specifica la data desiderata come "YYYY-MM-DD" (31 dicembre dell'anno specificato)
        datainizio = f"{year}-12-20"
        datafine = f"{year}-12-31"

        # Converti la data desiderata in un oggetto datetime
        datainizio = datetime.datetime.strptime(datainizio, "%Y-%m-%d").date()
        datafine = datetime.datetime.strptime(datafine, "%Y-%m-%d").date()


        try:
            val = yq_ticker.history(start=datainizio, end=datafine)['close'][-1]
        except Exception as e:
            print(e.__cause__)
            val = "NaN"
            tickers_non_scaricati.append(ticker)

        print(ticker, val)
        valori_borsa.append(val)



MARR 11.399999618530273
MARR 18.899999618530273
MARR 16.860000610351562
MARR 20.350000381469727
MARR 20.600000381469727
MARR 21.520000457763672
MARR 17.350000381469727
MARR 19.079999923706055
MARR 14.720000267028809
MARR 12.079999923706055
PIA 2.803999900817871
PIA 2.874000072479248
PIA 2.696000099182129
PIA 2.747999906539917
PIA 1.8309999704360962
PIA 2.299999952316284
PIA 1.5859999656677246
PIA 2.3259999752044678
PIA 2.4100000858306885
PIA 2.4119999408721924
BRE 10.449999809265137
BRE 12.529999732971191
BRE 10.788844108581543
BRE 11.0600004196167
BRE 8.895000457763672
BRE 12.670000076293945
BRE 11.5
BRE 8.935999870300293
BRE 5.539999961853027
BRE 3.9159998893737793
DAN 20.799999237060547
DAN 27.049999237060547
DAN 14.380000114440918
DAN 16.3799991607666
DAN 15.260000228881836
DAN 19.809999465942383
DAN 19.309999465942383
DAN 17.600000381469727
DAN 20.579999923706055
DAN 24.950000762939453
BC 69.0999984741211
BC 60.70000076293945
BC 35.70000076293945
BC 31.559999465942383
BC 30.049999

In [24]:
print(len(valori_borsa))
print(set(tickers_non_scaricati))

280
{'PLT', 'ILP', 'ITD', 'MDC', 'MARP', 'ALA', 'SVS', 'FOS', 'VIM', 'SCK', 'ICF', 'GPI', 'LDB', 'FCM', 'FVI', 'MASI', 'STAR7', 'CFL'}


## crea nuovo df
1. prendi info di interesse, moltiplica per ogni anno ogni osservazione 
2. aggiungi gli anni di interesse
3. doppio ciclo for: per ogni riga df originale prendi i primi nove valori e trasponi il risultato, passa ai prossimi nove (secondo ciclo)

#### crea df delle info 

In [25]:
info = pd.concat([df.iloc[:, 0:5]] * 10)
info.sort_index(inplace=True)
info

,ID,Ragione sociale,Ticker symbol,ISIN number,Numero CCIAA
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618
...,...,...,...,...,...
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153


#### crea colonna degli anni, aggiungi a info

In [26]:
anni = [anno for anno in range(2022, 2012, -1)] * 28
info['Anni'] = anni
info

,ID,Ragione sociale,Ticker symbol,ISIN number,Numero CCIAA,Anni
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018
...,...,...,...,...,...,...
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014


#### df degli indici

In [28]:
# Rimuovi il carattere '\n' e tutti quelli che seguono dalle intestazioni delle colonne
df.columns = df.columns.str.replace(r'\n.*', '', regex=True)
nomi_colonne = df.columns.unique()[5:]
# prendi df di interesse, serve a semplificare gli indici
indici = df.iloc[:, 5:]

# crea nuovo df
new_df = pd.DataFrame()

col = 0
#per ogni set di 10 valori riga di df originario INDICI
for i in range(0, 140, 10):
    
    indice = []
    #per ogni riga AZIENDA
    for j in range(1,len(indici)+1):
        # prendi i nove valori, trasponili e aggiungili al vettore dell'indice considerato
        valori = indici.iloc[j-1:j, i:i+10].values[0]
        valori = valori.T.tolist()
        
        indice= indice + valori
        
    # aggiungi colonna a new_df con insert
    new_df.insert(col, nomi_colonne[col], indice)
    col += 1


for col in new_df.columns:
    new_df[col] = new_df[col].str.replace(',' , '.').astype(float)
    #print(type(X[col]))

new_df

,Indice corrente,Indice di liquidità,Rotaz. cap. investito (volte),Rotaz. cap. cir. lordo (volte),Redditività del totale attivo (ROA),Redditività del totale attivo (ROA) (%),Redditività di tutto il capitale investito (ROI) (%),Redditività delle vendite (ROS) (%),Redditività del capitale proprio (ROE) (%),Indice di copertura delle immob. (patrimoniale),Costo denaro a prestito (%),Oneri finanz. su fatt. (%),Grado di copertura degli interessi passivi,Debiti v/banche su fatt. (%)
0,1.58,1.18,1.55,2.21,3.44,5.34,2.22,7.77,0.45,3.79,0.50,8.11,13.25,0.95
1,1.57,1.20,1.17,1.66,4.61,7.26,3.83,9.50,0.42,4.31,0.68,8.88,15.76,0.86
2,1.70,1.38,0.98,1.40,0.09,0.12,0.09,-1.25,0.37,1.60,0.57,5.90,35.37,1.29
3,1.46,1.16,1.46,2.04,8.84,13.31,5.88,19.42,0.33,2.15,0.40,19.02,18.59,1.04
4,1.67,1.34,1.54,2.06,9.37,14.36,5.92,20.32,0.20,1.87,0.35,20.30,18.72,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,3.34,3.31,0.02,0.04,6.48,7.48,22.97,5.28,0.03,1.42,1.13,25.64,79.55,0.35
276,2.39,2.37,0.02,0.03,6.51,8.08,24.26,8.55,0.02,2.01,1.26,25.00,62.68,0.27
277,2.55,2.53,0.02,0.03,7.92,9.94,29.06,13.71,0.02,3.10,1.53,23.10,49.29,0.20
278,2.23,2.21,0.02,0.03,7.20,9.28,25.64,12.34,0.02,9.10,2.33,13.82,25.64,0.10


#### aggiungi anche valori azionari

In [33]:
# aggiungi i valori della borsa al df info
info['Valori Borsa'] = valori_borsa
info

,ID,Ragione sociale,Ticker symbol,ISIN number,Numero CCIAA,Anni,Valori Borsa
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,11.4
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,18.9
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,16.860001
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,20.35
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,20.6
...,...,...,...,...,...,...,...
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,3.68
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,3.352
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,4.85
27,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,2.31


##### esegui il join dei due dataframe, resetta gli indici!!


In [34]:
#resetta indici
info = info.reset_index(drop=True)
new_df = new_df.reset_index(drop=True)
#esegui il join
dati_puliti = info.join(new_df)
dati_puliti


,ID,Ragione sociale,Ticker symbol,ISIN number,Numero CCIAA,Anni,Valori Borsa,Indice corrente,Indice di liquidità,Rotaz. cap. investito (volte),...,Redditività del totale attivo (ROA),Redditività del totale attivo (ROA) (%),Redditività di tutto il capitale investito (ROI) (%),Redditività delle vendite (ROS) (%),Redditività del capitale proprio (ROE) (%),Indice di copertura delle immob. (patrimoniale),Costo denaro a prestito (%),Oneri finanz. su fatt. (%),Grado di copertura degli interessi passivi,Debiti v/banche su fatt. (%)
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,11.4,1.58,1.18,1.55,...,3.44,5.34,2.22,7.77,0.45,3.79,0.50,8.11,13.25,0.95
1,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,18.9,1.57,1.20,1.17,...,4.61,7.26,3.83,9.50,0.42,4.31,0.68,8.88,15.76,0.86
2,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,16.860001,1.70,1.38,0.98,...,0.09,0.12,0.09,-1.25,0.37,1.60,0.57,5.90,35.37,1.29
3,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,20.35,1.46,1.16,1.46,...,8.84,13.31,5.88,19.42,0.33,2.15,0.40,19.02,18.59,1.04
4,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,20.6,1.67,1.34,1.54,...,9.37,14.36,5.92,20.32,0.20,1.87,0.35,20.30,18.72,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,3.68,3.34,3.31,0.02,...,6.48,7.48,22.97,5.28,0.03,1.42,1.13,25.64,79.55,0.35
276,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,3.352,2.39,2.37,0.02,...,6.51,8.08,24.26,8.55,0.02,2.01,1.26,25.00,62.68,0.27
277,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,4.85,2.55,2.53,0.02,...,7.92,9.94,29.06,13.71,0.02,3.10,1.53,23.10,49.29,0.20
278,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,2.31,2.23,2.21,0.02,...,7.20,9.28,25.64,12.34,0.02,9.10,2.33,13.82,25.64,0.10


##### ripulisci i dati, togli le aziende che non hanno valori di brosa per almeno un anno e calcola gli indici per ogni azienda

In [37]:
# elimina le aziende tramite una maschera
tickers_non_scaricati = list(set(tickers_non_scaricati))
maschera = dati_puliti['Ticker symbol'].isin(tickers_non_scaricati)
df_pulito = dati_puliti[~maschera]

df_pulito

,ID,Ragione sociale,Ticker symbol,ISIN number,Numero CCIAA,Anni,Valori Borsa,Indice corrente,Indice di liquidità,Rotaz. cap. investito (volte),...,Redditività del totale attivo (ROA),Redditività del totale attivo (ROA) (%),Redditività di tutto il capitale investito (ROI) (%),Redditività delle vendite (ROS) (%),Redditività del capitale proprio (ROE) (%),Indice di copertura delle immob. (patrimoniale),Costo denaro a prestito (%),Oneri finanz. su fatt. (%),Grado di copertura degli interessi passivi,Debiti v/banche su fatt. (%)
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,11.4,1.58,1.18,1.55,...,3.44,5.34,2.22,7.77,0.45,3.79,0.50,8.11,13.25,0.95
1,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,18.9,1.57,1.20,1.17,...,4.61,7.26,3.83,9.50,0.42,4.31,0.68,8.88,15.76,0.86
2,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,16.860001,1.70,1.38,0.98,...,0.09,0.12,0.09,-1.25,0.37,1.60,0.57,5.90,35.37,1.29
3,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,20.35,1.46,1.16,1.46,...,8.84,13.31,5.88,19.42,0.33,2.15,0.40,19.02,18.59,1.04
4,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,20.6,1.67,1.34,1.54,...,9.37,14.36,5.92,20.32,0.20,1.87,0.35,20.30,18.72,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,3.68,3.34,3.31,0.02,...,6.48,7.48,22.97,5.28,0.03,1.42,1.13,25.64,79.55,0.35
276,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,3.352,2.39,2.37,0.02,...,6.51,8.08,24.26,8.55,0.02,2.01,1.26,25.00,62.68,0.27
277,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,4.85,2.55,2.53,0.02,...,7.92,9.94,29.06,13.71,0.02,3.10,1.53,23.10,49.29,0.20
278,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,2.31,2.23,2.21,0.02,...,7.20,9.28,25.64,12.34,0.02,9.10,2.33,13.82,25.64,0.10


In [38]:
#calcola per ogni azienda con groupby gli indici
df_pulito['valore_2013'] = np.repeat(df_pulito[df_pulito['Anni'] == 2013]['Valori Borsa'].values, 10)
df_pulito['rapporto'] = df_pulito['Valori Borsa'] / df_pulito['valore_2013'] - 1
# decidi un threshold con cui assegnare zero e uno
treshold = 0.4
df_pulito['More Than'] = [0 if valore < treshold else 1 for valore in df_pulito['rapporto']]
df_pulito['More Than']

/tmp/ipykernel_10962/1450624068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pulito['valore_2013'] = np.repeat(df_pulito[df_pulito['Anni'] == 2013]['Valori Borsa'].values, 10)
/tmp/ipykernel_10962/1450624068.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pulito['rapporto'] = df_pulito['Valori Borsa'] / df_pulito['valore_2013'] - 1
/tmp/ipykernel_10962/1450624068.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

0      0
1      1
2      0
3      1
4      1
      ..
275    1
276    1
277    1
278    0
279    0
Name: More Than, Length: 100, dtype: int64

### salva il nuovo df 


In [39]:
df_pulito.to_csv("dati_puliti.csv")

# CREA IL MODELLO


In [40]:
df = pd.read_csv('/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/dati_puliti.csv', sep=',')
df

,Unnamed: 0,ID,Ragione sociale,Ticker symbol,ISIN number,Numero CCIAA,Anni,Valori Borsa,Indice corrente,Indice di liquidità,...,Redditività delle vendite (ROS) (%),Redditività del capitale proprio (ROE) (%),Indice di copertura delle immob. (patrimoniale),Costo denaro a prestito (%),Oneri finanz. su fatt. (%),Grado di copertura degli interessi passivi,Debiti v/banche su fatt. (%),valore_2013,rapporto,More Than
0,0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,11.400000,1.58,1.18,...,7.77,0.45,3.79,0.50,8.11,13.25,0.95,12.080,-0.056291,0
1,1,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,18.900000,1.57,1.20,...,9.50,0.42,4.31,0.68,8.88,15.76,0.86,12.080,0.564570,1
2,2,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,16.860001,1.70,1.38,...,-1.25,0.37,1.60,0.57,5.90,35.37,1.29,12.080,0.395695,0
3,3,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,20.350000,1.46,1.16,...,19.42,0.33,2.15,0.40,19.02,18.59,1.04,12.080,0.684603,1
4,4,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,20.600000,1.67,1.34,...,20.32,0.20,1.87,0.35,20.30,18.72,0.94,12.080,0.705298,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,275,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,3.680000,3.34,3.31,...,5.28,0.03,1.42,1.13,25.64,79.55,0.35,2.322,0.584841,1
96,276,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,3.352000,2.39,2.37,...,8.55,0.02,2.01,1.26,25.00,62.68,0.27,2.322,0.443583,1
97,277,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,4.850000,2.55,2.53,...,13.71,0.02,3.10,1.53,23.10,49.29,0.20,2.322,1.088717,1
98,278,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,2.310000,2.23,2.21,...,12.34,0.02,9.10,2.33,13.82,25.64,0.10,2.322,-0.005168,0


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [43]:
# Separazione delle variabili di input (X) e output (y)
X = df.iloc[:, 8:-3]
y = df['More Than']

X

,Indice corrente,Indice di liquidità,Rotaz. cap. investito (volte),Rotaz. cap. cir. lordo (volte),Redditività del totale attivo (ROA),Redditività del totale attivo (ROA) (%),Redditività di tutto il capitale investito (ROI) (%),Redditività delle vendite (ROS) (%),Redditività del capitale proprio (ROE) (%),Indice di copertura delle immob. (patrimoniale),Costo denaro a prestito (%),Oneri finanz. su fatt. (%),Grado di copertura degli interessi passivi,Debiti v/banche su fatt. (%)
0,1.58,1.18,1.55,2.21,3.44,5.34,2.22,7.77,0.45,3.79,0.50,8.11,13.25,0.95
1,1.57,1.20,1.17,1.66,4.61,7.26,3.83,9.50,0.42,4.31,0.68,8.88,15.76,0.86
2,1.70,1.38,0.98,1.40,0.09,0.12,0.09,-1.25,0.37,1.60,0.57,5.90,35.37,1.29
3,1.46,1.16,1.46,2.04,8.84,13.31,5.88,19.42,0.33,2.15,0.40,19.02,18.59,1.04
4,1.67,1.34,1.54,2.06,9.37,14.36,5.92,20.32,0.20,1.87,0.35,20.30,18.72,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.34,3.31,0.02,0.04,6.48,7.48,22.97,5.28,0.03,1.42,1.13,25.64,79.55,0.35
96,2.39,2.37,0.02,0.03,6.51,8.08,24.26,8.55,0.02,2.01,1.26,25.00,62.68,0.27
97,2.55,2.53,0.02,0.03,7.92,9.94,29.06,13.71,0.02,3.10,1.53,23.10,49.29,0.20
98,2.23,2.21,0.02,0.03,7.20,9.28,25.64,12.34,0.02,9.10,2.33,13.82,25.64,0.10


In [44]:

# Divisione del dataset in set di addestramento e di test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creazione e addestramento del modello di regressione logistica
model = LogisticRegression()
model.fit(X_train, y_train)

# Eseguire le previsioni sul set di test
y_pred = model.predict(X_test)

# Calcolare l'accuratezza del modello
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.75


/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/env_tesi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
